In [ ]:
import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.summarystats as ds
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from delfi.simulator.Gauss import Gauss


n_params = 1
seed = 42
m = Gauss(dim=n_params, seed=seed, noise_cov=1.)
p = dd.Gaussian(m=0. * np.ones(n_params), 
                S=1. *np.eye(n_params),
               seed=seed)
s = ds.Identity()
g = dg.Default(model=m, prior=p, summary=s)

# generate and plot
params, stats = g.gen(1000)
plt.plot(stats, params, 'b.');
plt.xlabel('x')
plt.ylabel('theta')

obs = 0.8 * np.ones((1,1)) #_,obs = g.gen(1)


In [ ]:
# set up inference
n_fits = 100

n_rounds = 5
n_train  = 5000

n_hiddens=[10]
svi=False

out_snpe   = np.zeros((n_fits,n_rounds, 2))
out_cdelfi = np.zeros((n_fits,n_rounds, 2))
for idx_seed in range(n_fits):
    
    seed = idx_seed + 42

    print('seed: ', seed)
    for algo in range(2):
        m = Gauss(dim=n_params, seed=seed, noise_cov=1.)
        p = dd.Gaussian(m=0. * np.ones(n_params), 
                        S=1. * np.eye(n_params),
                       seed=seed)
        s = ds.Identity()
        g = dg.Default(model=m, prior=p, summary=s)
        
        if algo==0:
            res = infer.CDELFI(g, n_hiddens=n_hiddens, seed=seed, svi=svi, obs=obs)
        elif algo==1:
            res = infer.SNPE(  g, n_hiddens=n_hiddens, seed=seed, svi=svi, obs=obs)

        # run with N samples
        for r in range(n_rounds):
            
            log, trn_data = res.run(n_train=n_train, n_rounds=1)
            posterior = res.predict(np.array([obs]).reshape(1,-1))
        
            if algo==0:
                out_cdelfi[idx_seed,r,:] = (posterior.mean, posterior.std**2)
            elif algo==1:
                out_snpe[idx_seed,r,:] = (posterior.mean, posterior.std**2)

        if algo==0:
            print('- CDELFI: \n', out_cdelfi[idx_seed,:,:])
        elif algo==1:
            print('- SNPE: \n', out_snpe[idx_seed,:,:])    
            
    np.save('res_tmp', {'out_cdelfi': out_cdelfi,
                        'out_snpe': out_snpe,
                        'n_fits': n_fits, 
                        'n_rounds': n_rounds, 
                        'n_train': n_train, 
                        'n_hiddens' : n_hiddens,
                        'svi' : svi,
                        'p_mu' : p.mean, 
                        'p_sig2' : p.std**2,
                        'noise_cov' : m.noise_cov}
           )
                

In [ ]:
mu_true = 0.4
sig_true = 0.5

for r in range(n_rounds):

    plt.figure(figsize=(12,9))

    plt.subplot(2,2,1)
    plt.hist(out_cdelfi[:,r,0], bins = np.linspace(0.3, 0.55, 20))
    plt.plot(mu_true*np.ones(2), [0, 14], 'r', linewidth=2)
    plt.ylabel('counts')
    plt.title('CDELFI')
    plt.legend(['analytic solution','outcome (n='+ str(n_fits)+')'], frameon=False)
    plt.subplot(2,2,2)
    plt.hist(out_cdelfi[:,r,1], bins = np.linspace(0.4, 0.65, 20))
    plt.plot(sig_true*np.ones(2), [0, 14], 'r', linewidth=2)
    plt.title('CDELFI')

    plt.subplot(2,2,3)
    plt.hist(out_snpe[:,r,0], bins = np.linspace(0.3, 0.55, 20))
    plt.title('SNPE')
    plt.ylabel('counts')
    plt.plot(mu_true*np.ones(2), [0, 14], 'r', linewidth=2)
    plt.xlabel('posterior mean')
    plt.subplot(2,2,4)
    plt.hist(out_snpe[:,r,1], bins = np.linspace(0.4, 0.65, 20))
    plt.plot(sig_true*np.ones(2), [0, 14], 'r', linewidth=2)
    plt.title('SNPE')
    plt.xlabel('posterior variance')

    #plt.savefig('1d_toy_case_20runs_SNPE_and_CDELFI_N5000_r3.pdf')

    plt.show()

In [ ]:
plt.figure(figsize=(12,8))


m_axis = np.minimum(out_cdelfi[:,:,0].min(), out_snpe[:,:,0].min())
M_axis = np.maximum(out_cdelfi[:,:,0].max(), out_snpe[:,:,0].max())
plt.subplot(2,2,1)
plt.plot(np.arange(n_rounds)+1, out_cdelfi[:,:,0].T, color=0.5*np.ones(3))
plt.plot(np.arange(n_rounds)+1, out_cdelfi[:,:,0].mean(axis=0), color='k')
plt.plot([1, n_rounds], mu_true*np.array([1,1]), 'r--')
plt.xticks(np.arange(n_rounds)+1)
plt.title('CDELFI')
plt.ylabel('posterior mean')
plt.axis([0.9, n_rounds+0.1, m_axis, M_axis])

plt.subplot(2,2,2)
plt.plot(np.arange(n_rounds)+1, out_snpe[:,:,0].T, color=0.5*np.ones(3))
plt.plot(np.arange(n_rounds)+1, out_snpe[:,:,0].mean(axis=0), color='k')
plt.title('SNPE')
plt.xticks(np.arange(n_rounds)+1)
plt.plot([1, n_rounds], mu_true*np.array([1,1]), 'r--')
plt.axis([0.9, n_rounds+0.1, m_axis, M_axis])

m_axis = np.minimum(out_cdelfi[:,:,1].min(), out_snpe[:,:,1].min())
M_axis = np.maximum(out_cdelfi[:,:,1].max(), out_snpe[:,:,1].max())

plt.subplot(2,2,3)
plt.plot(np.arange(n_rounds)+1, out_cdelfi[:,:,1].T, color=0.5*np.ones(3))
plt.plot(np.arange(n_rounds)+1, out_cdelfi[:,:,1].mean(axis=0), color='k')
plt.ylabel('posterior variance')
plt.xlabel('rounds')
plt.xticks(np.arange(n_rounds)+1)
plt.plot([1, n_rounds], sig_true*np.array([1,1]), 'r--')
plt.axis([0.9, n_rounds+0.1, m_axis, M_axis])

plt.subplot(2,2,4)
plt.plot(np.arange(n_rounds)+1, out_snpe[:,:,1].T, color=0.5*np.ones(3))
plt.plot(np.arange(n_rounds)+1, out_snpe[:,:,1].mean(axis=0), color='k')
plt.xlabel('rounds')
plt.xticks(np.arange(n_rounds)+1)
plt.plot([1, n_rounds], sig_true*np.array([1,1]), 'r--')
plt.axis([0.9, n_rounds+0.1, m_axis, M_axis])

plt.savefig('1d_toy_case_100runs_SNPE_and_CDELFI_N5000_noSVI.pdf')
np.save('1d_toy_case_100runs_SNPE_and_CDELFI_N5000_noSVI_resultDATA', 
        {'out_cdelfi': out_cdelfi,
         'out_snpe': out_snpe,
         'n_fits': n_fits, 
         'n_rounds': n_rounds, 
         'n_train': n_train, 
         'n_hiddens' : n_hiddens,
         'svi' : svi,
         'p_mu' : p.mean, 
         'p_sig2' : p.std**2,
         'noise_cov' : m.noise_cov})
plt.show()


In [ ]:
r=4
iws = trn_data[r][2]
(iws.min(), iws.max(), iws.mean())

In [ ]:
for r in range(1, len(trn_data)):
    plt.hist(trn_data[r][2])
    plt.show()